# Example of geometry optimization

In [5]:
import torch
from seqm.seqm_functions.constants import Constants
from seqm.Molecule import Molecule
from seqm.MolecularDynamics import Geometry_Optimization_SD
#from seqm.seqm_functions.read_xyz import read_xyz
import numpy as np
torch.set_default_dtype(torch.float64)
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

In [16]:
def read_xyz(files):
    '''
    reads xyz structure from a list (files) of files names
    '''
    #SPECIES = []
    COORDINATES = []
    for file in files:
        f = open(file)
        lines = f.readlines()
        f.close()
        Natoms = int(lines[0])
        #species = []
        coords = []
        for i in range(2, 2+Natoms):
            #species.append(int(lines[i].split()[0]))
            coords.append([int(lines[i].split()[0]), float(lines[i].split()[1]), float(lines[i].split()[2]), float(lines[i].split()[3])])
        #SPECIES.append(species)
        COORDINATES.append(coords)
    COORDINATES = np.array(COORDINATES)
    COORDINATES = COORDINATES[:,(-1*COORDINATES[:, :, 0]).argsort()]
    SPECIES =COORDINATES[:,:,:,0].astype(int)[0]
    COORDINATES = COORDINATES[:,:,:,1:4][0]

    return SPECIES, COORDINATES

In [17]:
%%time
torch.manual_seed(0)
files = ['coronene.xyz', 'coronene.xyz']

species, coordinates = read_xyz(files)
species = torch.as_tensor(species,dtype=torch.int64, device=device)[:]
coordinates = torch.tensor(coordinates, device=device)[:]
const = Constants().to(device)
#may need to add scaling factor for length and energy on const, check constants.py

elements = [0]+sorted(set(species.reshape(-1).tolist()))
seqm_parameters = {
                   'method' : 'AM1',  # AM1, MNDO, PM#
                   'scf_eps' : 1.0e-6,  # unit eV, change of electric energy, as nuclear energy doesnt' change during SCF
                   'scf_converger' : [2,0.0], # converger used for scf loop
                                         # [0, 0.1], [0, alpha] constant mixing, P = alpha*P + (1.0-alpha)*Pnew
                                         # [1], adaptive mixing
                                         # [2], adaptive mixing, then pulay
                   'sp2' : [False, 1.0e-5],  # whether to use sp2 algorithm in scf loop,
                                            #[True, eps] or [False], eps for SP2 conve criteria
                   'elements' : elements, #[0,1,6,8],
                   'learned' : [], # learned parameters name list, e.g ['U_ss']
                   #'parameter_file_dir' : '../seqm/params/', # file directory for other required parameters
                   'pair_outer_cutoff' : 1.0e10, # consistent with the unit on coordinates
                   'eig' : True
                   }

mol = Molecule(const, seqm_parameters, coordinates, species).to(device)

opt =  Geometry_Optimization_SD(seqm_parameters, alpha=0.003, force_tol=1.0e-1, max_evl=1000).to(device)
coordinates, force_err, energy_err =  opt.run(mol)

1 6.239775e-01 4.242794e+00 0.000000e+00 4.242794e+00 4.242794e+00 0.000000e+00 4.242794e+00 
2 4.805291e-01 4.213922e+00 4.242794e+00 -2.887210e-02 4.213922e+00 4.242794e+00 -2.887210e-02 
3 3.749148e-01 4.199006e+00 4.213922e+00 -1.491619e-02 4.199006e+00 4.213922e+00 -1.491619e-02 
4 2.951300e-01 4.190817e+00 4.199006e+00 -8.188616e-03 4.190817e+00 4.199006e+00 -8.188627e-03 
5 2.381131e-01 4.186020e+00 4.190817e+00 -4.797443e-03 4.186020e+00 4.190817e+00 -4.797433e-03 
6 1.943253e-01 4.183014e+00 4.186020e+00 -3.005341e-03 4.183014e+00 4.186020e+00 -3.005349e-03 
7 1.611150e-01 4.181025e+00 4.183014e+00 -1.989355e-03 4.181025e+00 4.183014e+00 -1.989353e-03 
8 1.344874e-01 4.179646e+00 4.181025e+00 -1.379087e-03 4.179646e+00 4.181025e+00 -1.379083e-03 
9 1.136134e-01 4.178655e+00 4.179646e+00 -9.904667e-04 4.178655e+00 4.179646e+00 -9.904622e-04 
10 9.542415e-02 4.177923e+00 4.178655e+00 -7.323228e-04 4.177923e+00 4.178655e+00 -7.323238e-04 
converged with 10 step, Max Force = 9.542

In [18]:
%%time
torch.manual_seed(0)
files = ['coronene.xyz']

species, coordinates = read_xyz(files)
species = torch.as_tensor(species,dtype=torch.int64, device=device)[:]
coordinates = torch.tensor(coordinates, device=device)[:]
const = Constants().to(device)
#may need to add scaling factor for length and energy on const, check constants.py

elements = [0]+sorted(set(species.reshape(-1).tolist()))
seqm_parameters = {
                   'method' : 'AM1',  # AM1, MNDO, PM#
                   'scf_eps' : 1.0e-6,  # unit eV, change of electric energy, as nuclear energy doesnt' change during SCF
                   'scf_converger' : [2,0.0], # converger used for scf loop
                                         # [0, 0.1], [0, alpha] constant mixing, P = alpha*P + (1.0-alpha)*Pnew
                                         # [1], adaptive mixing
                                         # [2], adaptive mixing, then pulay
                   'sp2' : [False, 1.0e-5],  # whether to use sp2 algorithm in scf loop,
                                            #[True, eps] or [False], eps for SP2 conve criteria
                   'elements' : elements, #[0,1,6,8],
                   'learned' : [], # learned parameters name list, e.g ['U_ss']
                   #'parameter_file_dir' : '../seqm/params/', # file directory for other required parameters
                   'pair_outer_cutoff' : 1.0e10, # consistent with the unit on coordinates
                   'eig' : True
                   }

mol = Molecule(const, seqm_parameters, coordinates, species).to(device)

opt =  Geometry_Optimization_SD(seqm_parameters, alpha=0.003, force_tol=1.0e-1, max_evl=1000).to(device)
coordinates,force_err, energy_err =  opt.run(mol)

1 6.239775e-01 4.242794e+00 0.000000e+00 4.242794e+00 
2 4.805291e-01 4.213922e+00 4.242794e+00 -2.887210e-02 
3 3.749148e-01 4.199006e+00 4.213922e+00 -1.491619e-02 
4 2.951300e-01 4.190817e+00 4.199006e+00 -8.188630e-03 
5 2.381125e-01 4.186020e+00 4.190817e+00 -4.797428e-03 
6 1.943251e-01 4.183014e+00 4.186020e+00 -3.005349e-03 
7 1.611147e-01 4.181025e+00 4.183014e+00 -1.989351e-03 
8 1.344874e-01 4.179646e+00 4.181025e+00 -1.379084e-03 
9 1.136134e-01 4.178655e+00 4.179646e+00 -9.904661e-04 
10 9.542397e-02 4.177923e+00 4.178655e+00 -7.323260e-04 
converged with 10 step, Max Force = 9.542397e-02 (eV/Ang), dE = -7.323260e-04 (eV)
CPU times: user 46.3 s, sys: 839 ms, total: 47.2 s
Wall time: 2.73 s


In [19]:
species, coordinates = read_xyz(files)
species, coordinates

(array([[6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6,
         6, 6, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]),
 array([[[-0.48230796,  0.        , -4.98583584],
         [-0.48230796,  1.34890014,  2.25923508],
         [-0.48230796,  0.        ,  2.51037571],
         [-0.48230796,  1.85193619,  0.92703549],
         [-0.48230796,  3.24595401,  0.63632275],
         [-0.48230796,  3.70291001, -0.65742944],
         [-0.48230796,  2.80071004, -1.75917115],
         [-0.48230796,  2.35401082, -4.15439425],
         [-0.48230796, -3.24595401, -3.11178288],
         [-0.48230796, -3.70291001, -1.81803069],
         [-0.48230796, -2.80071004, -0.71628898],
         [-0.48230796, -3.24595495,  0.63632256],
         [-0.48230796,  3.24595495, -3.11178269],
         [-0.48230796, -0.94877352,  1.44847563],
         [-0.48230796,  0.92794007, -0.15304247],
         [-0.48230796, -0.47539703,  0.10823311],
         [-0.48230796, -1.40333711, -0.97645449],
         [-0.48230796

In [21]:
mol.coordinates

tensor([[[-4.8231e-01, -8.9861e-04, -4.9805e+00],
         [-4.8231e-01,  1.3461e+00,  2.2546e+00],
         [-4.8231e-01,  8.9861e-04,  2.5050e+00],
         [-4.8231e-01,  1.8466e+00,  9.2075e-01],
         [-4.8231e-01,  3.2418e+00,  6.3287e-01],
         [-4.8231e-01,  3.6975e+00, -6.5737e-01],
         [-4.8231e-01,  2.7926e+00, -1.7577e+00],
         [-4.8231e-01,  2.3513e+00, -4.1497e+00],
         [-4.8231e-01, -3.2418e+00, -3.1083e+00],
         [-4.8231e-01, -3.6975e+00, -1.8181e+00],
         [-4.8231e-01, -2.7926e+00, -7.1780e-01],
         [-4.8231e-01, -3.2409e+00,  6.3442e-01],
         [-4.8231e-01,  3.2409e+00, -3.1099e+00],
         [-4.8231e-01, -9.4602e-01,  1.4407e+00],
         [-4.8231e-01,  9.2999e-01, -1.5065e-01],
         [-4.8231e-01, -4.7645e-01,  1.1121e-01],
         [-4.8231e-01, -1.4064e+00, -9.7588e-01],
         [-4.8231e-01,  1.4064e+00, -1.4996e+00],
         [-4.8231e-01, -1.3461e+00, -4.7300e+00],
         [-4.8231e-01, -1.8466e+00, -3.3962e+00],
